In [11]:
from functools import *
import random

In [12]:
# Se le pasa una lista que define la permutación
def pi_inverse(l):
    n = len(l)
    Bn = BraidGroup(n)
    p = PermutationGroupElement(l)
    ciclos = p.cycle_tuples()
    b = Bn([1,-1])
    
    
    # Descomponemos en transposiciones los ciclos
    desc_trans = []
    for i in range(len(ciclos)):
        for j in range(len(ciclos[i])-1,0, -1):
            desc_trans.append((ciclos[i][0], ciclos[i][j]))
    
    desc_trans_sim = []
    for i in range(len(desc_trans)):
        d = [(desc_trans[i][0],desc_trans[i][0]+1)]
        for j in range(desc_trans[i][0]+1,desc_trans[i][1]):
            d = [(j,j+1)] + d + [(j,j+1)]
        desc_trans_sim = desc_trans_sim + d
            
    for i in range(len(desc_trans_sim)-1,-1,-1):          
                b = b * Bn([desc_trans_sim[i][0]])

        
    return b
    
    

In [13]:
def GuessPermutation(v,w,n):
    Bn = BraidGroup(n)
    Sn = SymmetricGroup(n)
    t = [i+1 for i in range(n)]
    
    chi = [False for i in range(n)]   
    for i in range(n,0,-1):
        r = i
        s = i
        while not chi[r-1]:
            chi[r-1] = True
            pi_v = Sn(v.permutation())
            #print(pi_v)
            r = pi_v(r)
            pi_w = Sn(w.permutation())
            #print(pi_w)
            s = pi_w(s)
            if r != s:
                t[r-1] = s
            #print(t)
    
    
    xi = [False for i in range(n)]
    for i in range(n,0,-1):
        if not xi[i-1]:
            xi[i-1] = True
            r=i
            while (not xi[t[r-1]-1]) and (t[r-1] != r):
                r = t[r-1]
                xi[r-1] = True
            t[r-1] = i
    
    print(t)
            
    return pi_inverse(t)
            
        
    

In [14]:
def order_cycles(l):
    mov = True

    while mov:
        mov = False
        for i in range(len(l)-1):
            if len(l[i]) < len(l[i+1]):
                aux = l[i]
                l[i] = l[i+1]
                l[i+1] = aux
                mov = True
    
    return l

In [15]:
def GuessPermutation(v,w,n):
    vp = PermutationGroupElement(v.permutation()).cycle_tuples(singletons=True)
    wp = PermutationGroupElement(w.permutation()).cycle_tuples(singletons=True)
    
    vp = order_cycles(vp)
    wp = order_cycles(wp)
    

    
    tau = [i for i in range(n)]
    
    for i in range(len(wp)):
        for j in range(len(wp[i])):
            tau[vp[i][j]-1] = wp[i][j]
    print(tau)
    return pi_inverse(tau)
    

In [16]:
def inf(b_normal,n):
    Bn = BraidGroup(n)
    alpha = Bn([1,-1])
    delta = Bn.delta()
    neutro = Bn([1,-1])

    
    rb = 0
    
    delta_r = b_normal[0]
    while delta_r != neutro:
        delta_r = delta_r*delta
        rb = rb + 1
    
    return rb
    

In [17]:
def tauC(w,r,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    
    return (delta^-r)*w*(delta^r)

In [18]:
def compact(normal_form, n):
    Bn = BraidGroup(n)
    c = Bn([1,-1])
    
    for i in range(len(normal_form)):
        c = c*normal_form[i]
    
    return c
    


In [19]:
def breakConjugate(v,w,n):
    Bn = BraidGroup(n)
    delta = Bn.delta()
    alpha = Bn([1,-1])
    v_normal = v.left_normal_form()
    w_normal = w.left_normal_form()


    
    rw = inf(w_normal,n)
    rv = inf(v_normal,n)
    
    while rw < rv:
        gamma = delta*tauC(w_normal[1],rw,n)
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        rw = inf(w_normal,n)

    
    sw = len(w_normal)-1 + inf(w_normal,n)
    sv = len(v_normal)-1 + inf(v_normal,n)
    
    while sw > sv:
        gamma = w_normal[len(w_normal)-1]
        alpha = gamma*alpha
        w = gamma*compact(w_normal,n)*(gamma^-1)
        w_normal = w.left_normal_form()
        sw = len(w_normal)-1 + inf(w_normal,n)
    
    v = compact(v_normal,n)
    w = compact(w_normal,n)
    mu = GuessPermutation(v,w,n)
    alpha = mu*alpha
    w = mu*w*(mu^-1)
    
    if v == w:
        return alpha
    else:
        return "failed"
        
        
        
    
    
    
    
    

In [20]:
Bn = BraidGroup(5)
v = Bn([3,3,2,4,1,2])
x = Bn([3,1,4,2,3,4])
w = (x^-1)*v*x
conjugator = breakConjugate(v,w,5)
print(w == conjugator^-1*v*conjugator)
print(conjugator)
print(w.left_normal_form())
print((conjugator^-1*v*conjugator).left_normal_form())

[1, 2, 4, 3, 5]
True
s2*s0*s1*s2*s1*s0*s3*s2*s0*s1*s2*s3
(s0^-1*s1^-1*s2^-1*s3^-1*s0^-1*s1^-1*s2^-1*s0^-1*s1^-1*s0^-1, s0*s1*s2*s1*s0*s3*s2*s1*s0, s1*s0*s2, s0*s1*s2*s3)
(s0^-1*s1^-1*s2^-1*s3^-1*s0^-1*s1^-1*s2^-1*s0^-1*s1^-1*s0^-1, s0*s1*s2*s1*s0*s3*s2*s1*s0, s1*s0*s2, s0*s1*s2*s3)
